In [17]:
import numpy as np
import matplotlib.pyplot as plt
from CAMB import camb

In [23]:
wmap = np.loadtxt('Intensity.txt')

In [52]:
def get_spectrum(pars,lmax=2000,fixed_Tau=None):
    if fixed_Tau is None:
        H0=pars[0]
        ombh2=pars[1]
        omch2=pars[2]
        tau=pars[3]
        As=pars[4]
        ns=pars[5]
    else: 
        H0=pars[0]
        ombh2=pars[1]
        omch2=pars[2]
        tau=fixed_Tau
        As=pars[3]
        ns=pars[4]
    pars=camb.CAMBparams()
    pars.set_cosmology(H0=H0,ombh2=ombh2,omch2=omch2,mnu=0.06,omk=0,tau=tau)
    pars.InitPower.set_params(As=As,ns=ns,r=0)
    pars.set_for_lmax(lmax,lens_potential_accuracy=0)
    results=camb.get_results(pars)
    powers=results.get_cmb_power_spectra(pars,CMB_unit='muK')
    cmb=powers['total']
    tt=cmb[:,0]    #you could return the full power spectrum here if you wanted to do say EE
    return tt

In [41]:
opticalDepth = 0.05

In [154]:
def derivative(x,y,p,dx,fixed_Tau=None):
    """
    Takes the derivative of the function y (which is the spectrum) in terms 
    of each of its parameters. This is done by varying each parameter by
    a certain dx defined below. 
    """
    grad = np.zeros([len(x),len(p)])
    p_dx = p.copy()
    for i in range(len(p)):
        d_p = p_dx[i]*dx #small change in the parameter in question
        p_dx[i] = p_dx[i] + d_p #Adding that small change to the parameter
        spect_dx = y(p_dx)[2:len(x)+2]
        
        p_dx[i] = p_dx[i] - 2*d_p #small change int the other direction
        spect_dx_2 = y(p_dx)[2:len(x)+2]
        
        deriv = (spect_dx-spect_dx_2)/(2*d_p)
        grad[:,i] = np.asarray(deriv)
        p_dx = p.copy()
    return grad

def newton_Method(x,y,p,g,errP):
    chi_sqr = 10000
    l = 0.0001
    for j in range(300):
        guessing = y(p)[2:len(x)+2]
        residual = x - guessing
        chi_sqr_new = (residual**2/errP**2).sum()
        cond = chi_sqr - chi_sqr_new
        if cond<0.001 and cond>0:
            N = np.diag(errP**2)
            cov = np.dot(g.transpose(),N)
            cov = np.dot(cov,g)
            cov = np.linalg.inv(cov)
            err = np.sqrt(np.diag(cov))
            break
        elif chi_sqr_new > chi_sqr: 
            l = 5*l
            print ("Chi-squared got bigger, I just increased Lambda!")
        else:
            chi_sqr = chi_sqr_new
            l = l/3
            
        residual = np.matrix(residual).transpose()    
        lhs = g.transpose()*g
        lhs += l*np.diagonal(lhs)
        rhs = g.transpose()*residual
        changeP = np.linalg.inv(lhs)*rhs
        print (changeP,len(p))
        for k in range(len(p)):
            p[k] = p[k] + changeP[k]
        print (f"We are currently in iteration {j}; the chi-squared is {chi_sqr}")
    return p,cov,err

In [155]:
tau = opticalDepth
pars=np.asarray([65,0.02,0.1,2e-9,0.96])

In [140]:
fit = lambda pars: get_spectrum(pars,fixed_Tau=tau) 
g = derivative(wmap[:,1],fit,pars,0.001)
g = np.matrix(g)

In [142]:
p, pcov, err = newton_Method(wmap[:,1],PP,pars,g,wmap[:,2])

We are currently in iteration 0; the chi-squared is 1588.2376532931526
We are currently in iteration 1; the chi-squared is 1245.7982925681285
We are currently in iteration 2; the chi-squared is 1241.880377445329
We are currently in iteration 3; the chi-squared is 1241.6502266229793
We are currently in iteration 4; the chi-squared is 1241.6346290193499


In [143]:
print (p)
print (err)

[7.20845448e+01 2.33380399e-02 1.05405737e-01 1.99999977e-09
 9.98017981e-01]
[1.29502500e-05 4.52375377e-09 3.07023097e-08 1.72038287e-16
 6.63880144e-08]


The hubble constant $H_o$ was found to be equal to $72.08454$ $\pm$ $0.00001$, the physical baryon density to be $0.023$ $\pm$ $4.5\times 10^{-9}$, the cold dark matter density to be $0.1054$ $\pm$ $3\times 10^{-8}$, the primordial power law to be $1.999\times 10^{-9}$ $\pm$ $2\times 10^{-16}$, and finally the slope of the primordial law to be $0.998$ $\pm$ $\times 10^{-8}$

In [156]:
fit_tau = lambda tau: get_spectrum(p,fixed_Tau=tau)
new_tau, new_tau_cov,new_tau_error = newton_Method(wmap[:,1],fit_tau,np.asarray([tau]),g,wmap[:,2])


[[ 1.11140906e-03]
 [ 4.14648441e-07]
 [-6.14364574e-07]
 [-1.85943383e-18]
 [ 7.93665401e-06]] 1
We are currently in iteration 0; the chi-squared is 1241.633863919817
[[ 4.87306823e-01]
 [ 6.58929472e-05]
 [-8.23742621e-04]
 [-5.82688397e-18]
 [ 2.62933613e-03]] 1
We are currently in iteration 1; the chi-squared is 1241.5217195813066
Chi-squared got bigger, I just increased Lambda!
[[ 1.36166642e+02]
 [ 1.89379540e-02]
 [-2.30971850e-01]
 [-4.75850892e-18]
 [ 7.61629173e-01]] 1
We are currently in iteration 2; the chi-squared is 1241.5217195813066


CAMBError: Error in Fortran called from calc_transfer:
Reionization did not converge to optical depth